In [6]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import pandas as pd
import gensim
import pyLDAvis.gensim


In [ ]:
print(get_stop_words)

In [7]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = get_stop_words('en')
lemmatizer = WordNetLemmatizer()

In [8]:
df = pd.read_excel("D:\Projects\MRC\wipoLinking_ELECT_IntegreatedCircuits_test.xlsx")

<input>:1: DeprecationWarning: invalid escape sequence \P
<input>:1: DeprecationWarning: invalid escape sequence \P
<input>:1: DeprecationWarning: invalid escape sequence \P
<ipython-input-8-d64d8fbeb172>:1: DeprecationWarning: invalid escape sequence \P
  df = pd.read_excel("D:\Projects\MRC\wipoLinking_ELECT_IntegreatedCircuits_test.xlsx")


In [9]:
df

,TERM,Title,Text
0,fully-depleted silicon-on-insulator transistor,BUILDING FULLY-DEPLETED AND PARTIALLY-DEPLETED...,There are two types of conventional SOI device...
1,semiconductor substrate,SEMICONDUCTOR COMPONENT AND A METHOD FOR IDENT...,The present invention relates to a method for ...
2,transistor-transistor logic,IDENTIFICATION TAG,TTL - Transistor-Transistor Logic is a class o...
3,TTL,NaN,"TTL, i.e. transistor-transistor logic, is a co..."
4,land,ELECTRONIC CONTROL DEVICE FOR A LIGHTING DEVIC...,"The term ""land"" as used herein refers to that ..."
5,pad,NaN,The term pad as used herein typically refers t...
6,land pad,NaN,[...] wherein the printed circuit board has a ...
7,programmable chip,LOOK UP TABLE STRUCTURE SUPPORTING QUATERNARY ...,Programmable chips such as programmable logic ...
8,LGA package,MULTIPLE CHIP PACKAGE MODULE HAVING INVERTED P...,[T]he additional package can be a BCC or a QFN...
9,pin grid array package,METHOD AND APPARATUS FOR INSPECTION OF PIN GR...,"In the manufacturing of integrated circuits, t..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 3 columns):
TERM     58 non-null object
Title    43 non-null object
Text     58 non-null object
dtypes: object(3)
memory usage: 1.4+ KB


In [14]:
df = df[['TERM','Text']]

In [15]:
df

,TERM,Text
0,fully-depleted silicon-on-insulator transistor,There are two types of conventional SOI device...
1,semiconductor substrate,The present invention relates to a method for ...
2,transistor-transistor logic,TTL - Transistor-Transistor Logic is a class o...
3,TTL,"TTL, i.e. transistor-transistor logic, is a co..."
4,land,"The term ""land"" as used herein refers to that ..."
5,pad,The term pad as used herein typically refers t...
6,land pad,[...] wherein the printed circuit board has a ...
7,programmable chip,Programmable chips such as programmable logic ...
8,LGA package,[T]he additional package can be a BCC or a QFN...
9,pin grid array package,"In the manufacturing of integrated circuits, t..."


In [17]:
remove_words = ['data','dataset','datasets','content','context','acknowledgement','inspiration']

In [18]:
texts = []

# loop through document list
for i in df['Text'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [raw for raw in tokens if not raw in en_stop]
    
    # remove stop words from tokens
    stopped_tokens_new = [raw for raw in stopped_tokens if not raw in remove_words]
    
    # lemmatize tokens
    lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in stopped_tokens_new]
    
    # remove word containing only single char
    new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    
    # add tokens to list
    texts.append(new_lemma_tokens)

# sample data
print(texts[0])

['two', 'type', 'conventional', 'soi', 'device', 'partially', 'depleted', 'soi', 'pd', 'soi', 'device', 'fully', 'depleted', 'fd', 'soi', 'device', 'conventional', 'pd', 'soi', 'transistor', 'device', 'one', 'body', 'thickness', 'thicker', 'maximum', 'width', 'depletion', 'layer', 'silicon', 'device', 'operation', 'conventional', 'fd', 'soi', 'transistor', 'device', 'one', 'body', 'thickness', 'thinner', 'maximum', 'width', 'depletion', 'layer', 'silicon', 'device', 'operation', 'consequently', 'operation', 'pd', 'soi', 'device', 'experience', 'partial', 'depletion', 'silicon', 'body', 'fd', 'soi', 'experience', 'full', 'depletion', 'thereof', 'conventional', 'pd', 'soi', 'fd', 'soi', 'device', 'planar', 'device', 'thus', 'formed', 'plane', 'wafer', 'particularly', 'fully', 'depleted', 'silicon', 'insulator', 'fd', 'soi', 'transistor', 'can', 'formed', 'region', 'comprising', 'thinned', 'silicon', 'partially', 'depleted', 'silicon', 'insulator', 'pd', 'soi', 'transistor', 'can', 'forme

In [19]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [20]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, passes=20)
import pprint
pprint.pprint(ldamodel.top_topics(corpus,topn=5))

[([(0.047284763, 'event'),
   (0.027446462, 'test'),
   (0.024965487, 'scan'),
   (0.024964519, 'system'),
   (0.022486756, 'device')],
  -0.5950642552145478),
 ([(0.02933868, 'circuit'),
   (0.023776362, 'semiconductor'),
   (0.022431819, 'device'),
   (0.021656085, 'integrated'),
   (0.019793727, 'substrate')],
  -0.9703188334150219),
 ([(0.032431033, 'array'),
   (0.028866246, 'grid'),
   (0.028865092, 'pin'),
   (0.024323463, 'inspection'),
   (0.018268256, 'lead')],
  -1.1330603907818608),
 ([(0.041290045, 'semiconductor'),
   (0.02342808, 'substrate'),
   (0.023425862, 'deposition'),
   (0.022021085, 'first'),
   (0.018257786, 'region')],
  -1.506255482459129),
 ([(0.031219512, 'ball'),
   (0.025067993, 'solder'),
   (0.025064245, 'substrate'),
   (0.025056232, 'carrier'),
   (0.018901426, 'opening')],
  -1.552562603771682),
 ([(0.076266624, 'chip'),
   (0.06472856, 'system'),
   (0.02485637, 'component'),
   (0.020215265, 'measurement'),
   (0.020212654, 'soc')],
  -1.7852674870

In [21]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\Pawan\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.113343  0.147103       1        1  12.091547
7     -0.160597  0.075695       2        1  10.806782
9      0.001060  0.026386       3        1   9.938105
11    -0.020313  0.148262       4        1   9.705359
1     -0.013032  0.029986       5        1   9.514984
12    -0.107051 -0.115139       6        1   8.660734
5      0.169806 -0.000380       7        1   8.216057
2      0.111347 -0.023358       8        1   7.835020
4     -0.063283 -0.142827       9        1   7.231523
10     0.113670  0.038997      10        1   5.290989
13    -0.069293 -0.154784      11        1   5.039349
14     0.046513 -0.019748      12        1   2.537659
6      0.087669 -0.000880      13        1   1.866489
8     -0.028584  0.003166      14        1   1.199936
0      0.045432 -0.012480      15        1   0.065468, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
214   Default  27.000000           chip  27.000000  30.0000  30.0000
419   Default  27.000000         system  27.000000  29.0000  29.0000
1339  Default  26.000000        polygon  26.000000  28.0000  28.0000
42    Default  56.000000      substrate  56.000000  27.0000  27.0000
676   Default  16.000000          event  16.000000  26.0000  26.0000
131   Default  45.000000          first  45.000000  25.0000  25.0000
37    Default  45.000000  semiconductor  45.000000  24.0000  24.0000
209   Default  43.000000          array  43.000000  23.0000  23.0000
4     Default  93.000000        circuit  93.000000  22.0000  22.0000
269   Default  30.000000           grid  30.000000  21.0000  21.0000
88    Default  23.000000           mean  23.000000  20.0000  20.0000
279   Default  40.000000        package  40.000000  19.0000  19.0000
23    Default  42.000000          layer  42.000000  18.0000  18.0000
1383  Default  15.000000          laser  15.000000  17.0000  17.0000
187   Default  19.000000         solder  19.000000  16.0000  16.0000
57    Default  30.000000      component  30.000000  15.0000  15.0000
567   Default  30.000000           cell  30.000000  14.0000  14.0000
87    Default  35.000000       material  35.000000  13.0000  13.0000
373   Default  21.000000           test  21.000000  12.0000  12.0000
351   Default  21.000000            pin  21.000000  11.0000  11.0000
332   Default  14.000000     inspection  14.000000  10.0000  10.0000
293   Default  21.000000           wire  21.000000   9.0000   9.0000
149   Default  32.000000         second  32.000000   8.0000   8.0000
40    Default  12.000000            soi  12.000000   7.0000   7.0000
52    Default  35.000000          wafer  35.000000   6.0000   6.0000
380   Default   8.000000           ball   8.000000   5.0000   5.0000
545   Default  18.000000          plate  18.000000   4.0000   4.0000
265   Default  42.000000            die  42.000000   3.0000   3.0000
49    Default  26.000000     transistor  26.000000   2.0000   2.0000
164   Default  16.000000     conductive  16.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
567   Topic15   0.002688           cell  30.056026  -1.9907  -7.2565
209   Topic15   0.002688          array  43.290047  -2.3557  -7.2567
131   Topic15   0.002687          first  45.821712  -2.4126  -7.2567
279   Topic15   0.002687        package  40.466488  -2.2884  -7.2568
1339  Topic15   0.002686        polygon  26.746601  -1.8747  -7.2572
52    Topic15   0.002685          wafer  35.854984  -2.1681  -7.2575
1357  Topic15   0.002684         custom   6.703073  -0.4917  -7.2580
460   Topic15   0.002683             ic  26.057575  -1.8496  -7.2582
149   Topic15   0.002682         second  32.269604  -2.0639  -7.2586
293   Topic15   0.002682           wire  21.338947  -1.6505  -7.2588
10    Topic15   0.002681         device  66.279472  -2.7842  -7.2592
23 